## Q1. What is the difference between __getattr__ and __getattribute__?


Ans: __getattribute__ is used to find an attribute of a class. It raises an AttributeError of it fails to find an attribute of a class. __getattr__ is implemented latter if AttributeError is generated by __getattribute__, but for this __getattribute__ and __getattr__ both has to be defined in same class. If no attribute is found, __getattr__ returns a default value. So key difference is that __getattr__ is called for attributes that don't actually exist on a class.

In [1]:
class Example:
    def __init__(self):
        self.x = 1
        
    def __getattr__(self, name):
        print(f"__getattr__ called for {name}")
        if name == "y":
            return 2
        else:
            raise AttributeError(f"'{self.__class__.__name__}' object has no attribute '{name}'")
    
    def __getattribute__(self, name):
        print(f"__getattribute__ called for {name}")
        return super().__getattribute__(name)


In [2]:
e = Example()

In [3]:
e.x

__getattribute__ called for x


1

In [4]:
e.y

__getattribute__ called for y
__getattr__ called for y


2

In [5]:
e.z

__getattribute__ called for z
__getattr__ called for z
__getattribute__ called for __class__


AttributeError: 'Example' object has no attribute 'z'

## Q2. What is the difference between properties and descriptors?


In Python, properties and descriptors are both mechanisms for controlling access to an object's attributes. However, they work in slightly different ways.

A property is a built-in Python feature that allows you to define methods to get, set, or delete an object's attributes. When you define a property, you can access the attribute as if it were a normal attribute, but behind the scenes, the getter, setter, or deleter method is called.

In [44]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    @property
    def area(self):
        return self.width
    
    @area.setter
    def area(self, value):
        self.width = value
        
    

r = Rectangle(3, 4)
print(r.area) # Output
r.area = 15
r.area


3


15

A descriptor, on the other hand, is a more general mechanism for controlling attribute access. A descriptor is an object that defines one or more of the __get__, __set__, and __delete__ methods. When an attribute is accessed, Python checks whether the attribute has a descriptor. If it does, Python calls the appropriate method of the descriptor to get, set, or delete the attribute

In [36]:
class NonNegative:
    def __init__(self, name):
        self.name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self.name]

    def __set__(self, instance, value):
        if value < 0:
            raise ValueError("Value cannot be negative.")
        instance.__dict__[self.name] = value

class Person:
    age = NonNegative("age")

p = Person()
p.age = 30
print(p.age) # Output: 30
p.age = -10 # This will raise a ValueError


30


ValueError: Value cannot be negative.

## Q3. What are the key differences in functionality between getattr and getattribute, as well as properties and descriptors?


Ans: The Key Differences between __getattr__, __getattribute__, Properties and Descriptors are:

__getattr__: Python will call this method whenever you request an attribute that hasn't already been defined

__getattribute__ : This method will invoked before looking at the actual attributes on the object. Means,if we have __getattribute__ method in our class, python invokes this method for every attribute regardless whether it exists or not.

Properties: With Properties we can bind getter, setter and delete functions together with an attribute name, using the built-in property function or @property decorator. When we do this, each reference to an attribute looks like simple, direct access, but involes the appropriate function of the object.

Descriptor: With Descriptor we can bind getter, setter and delete functions into a seperate class. we then assign an object of this class to the attribute name in our main class. When we do this, each reference to an attribute looks like simple, direct access but invokes an appropriate function of descriptor object.

In [45]:
class ReadOnlyProperty:
    def __init__(self, getter):
        self.getter = getter
        
    def __get__(self, instance, owner):
        return self.getter(instance)

class Person:
    def __init__(self, name, age):
        self.name = name
        self._age = age
        
    @property
    def age(self):
        return self._age
    
    @ReadOnlyProperty
    def birth_year(self):
        return 2023 - self.age
        
p = Person("John", 30)

print(p.name)
print(p.age)
print(p.birth_year)

p.age = 31 # This will raise an AttributeError, because age is read-only
p.birth_year = 1993 # This will also raise an AttributeError, because birth_year is read-only


John
30
1993


AttributeError: can't set attribute